# Import Libraries

In [8]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

sns.set_context('talk')
mpl.rcParams['figure.figsize'] = [12,6]

In [9]:
pd.set_option('display.max_columns',100)

# Load Data

In [7]:
## read in the Data/student/student-mat.csv (it uses ";" as the sep)
df = pd.read_csv("Data/insurance - insurance.csv", sep=';')

# display info and .head
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 1 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   age,sex,bmi,children,smoker,region,charges  1338 non-null   object
dtypes: object(1)
memory usage: 10.6+ KB


,"age,sex,bmi,children,smoker,region,charges"
0,"19,female,27.9,0,yes,southwest,16884.924"
1,"18,male,33.77,1,no,southeast,1725.5523"
2,"28,male,33,3,no,southeast,4449.462"
3,"33,male,22.705,0,no,northwest,21984.47061"
4,"32,male,28.88,0,no,northwest,3866.8552"
...,...
1333,"50,male,30.97,3,no,northwest,10600.5483"
1334,"18,female,31.92,0,no,northeast,2205.9808"
1335,"18,female,36.85,0,no,southeast,1629.8335"
1336,"21,female,25.8,0,no,southwest,2007.945"


#  1. Do smokers have higher insurance charges than non-smokers?